In [4]:


#import
import os
import re
import csv
import shutil
import requests
import numpy as np
from pathlib import Path
from zipfile import ZipFile
from bs4 import BeautifulSoup


#pandas
import pandas as pd
import dask.dataframe as dd


#function
from _pd_utils import _folder_to_filestems, _lowercase, _tonumeric, _todatetime, \
    _groupby, _firstvalue_join_notna


#variable
from _census import statefips_dict, statenames_dict, statenames_list


#_beacagdp1 
#https://apps.bea.gov/regional/downloadzip.cfm, Gross domestic product (GDP): "CAGDP1"
#https://www.bea.gov/help/glossary
folders=["ciani/data/beacagdp1", "ciani/_beacagdp1"]
items=["CAGDP1__ALL_AREAS_2017_2022", "beacagdp1"]
def _beacagdp1(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "GeoFIPS",
        "LineCode",
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
        "2022",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        engine="python",
        dtype="string",
        skipinitialspace=True,
        skipfooter=4,
        na_values=["(NA)"],
        #nrows=100,
        encoding="latin-1",
        #on_bad_lines="skip",
        ) 
        
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "GeoFIPS",
        "LineCode",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs
    df=df[df["LineCode"]=="3"]
    df=df.reset_index(drop=True)

    #statefips
    df["statefips"]=df["GeoFIPS"].str[0:2]

    #countyfips
    df["countyfips"]=df["GeoFIPS"].str[2:5]

    #dropobs
    df=df[df["countyfips"]!="000"]
    df=df.reset_index(drop=True)

    #melt
    id_vars=[
        "GeoFIPS",
        ]
    value_vars=[
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
        "2022",
        ]
    var_name="year"
    value_name="gdp"
    df=pd.melt(
        frame=df,
        id_vars=id_vars,
        value_vars=value_vars,
        var_name=var_name,
        value_name=value_name,
        )

    #sortvalues 
    sortvalues_cols=[
        "GeoFIPS",
        "year",
        ]        
    df=df.sort_values(
        by=sortvalues_cols,
        )
    
    #ordered
    ordered_cols=[
        "GeoFIPS",
        "gdp",
        "year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_beacagdp1(folders, items)


#_beacainc1 
#https://apps.bea.gov/regional/downloadzip.cfm, Personal income (state and local): "CAINC1"
#https://www.bea.gov/help/glossary
folders=["ciani/data/beacainc1", "ciani/_beacainc1"]
items=["CAINC1__ALL_AREAS_1969_2022", "beacainc1"]
def _beacainc1(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "GeoFIPS",
        "LineCode",
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
        "2022",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        engine="python",
        dtype="string",
        skipinitialspace=True,
        skipfooter=4,
        na_values=["(NA)"],
        #nrows=100,
        encoding="latin-1",
        #on_bad_lines="skip",
        ) 
        
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "GeoFIPS",
        "LineCode",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs
    df=df[df["LineCode"]=="3"]
    df=df.reset_index(drop=True)

    #statefips
    df["statefips"]=df["GeoFIPS"].str[0:2]

    #countyfips
    df["countyfips"]=df["GeoFIPS"].str[2:5]

    #dropobs
    df=df[df["countyfips"]!="000"]
    df=df.reset_index(drop=True)

    #melt
    id_vars=[
        "GeoFIPS",
        ]
    value_vars=[
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
        "2022",
        ]
    var_name="year"
    value_name="inc"
    df=pd.melt(
        frame=df,
        id_vars=id_vars,
        value_vars=value_vars,
        var_name=var_name,
        value_name=value_name,
        )

    #sortvalues 
    sortvalues_cols=[
        "GeoFIPS",
        "year",
        ]        
    df=df.sort_values(
        by=sortvalues_cols,
        )
    
    #ordered
    ordered_cols=[
        "GeoFIPS",
        "inc",
        "year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_beacainc1(folders, items)


#_faers_extract
#https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html
folders=["ciani/data/faers", "ciani/_faers"]
items=[]
def _faers_extract(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    #result=items[0]

    #zipresults
    zipresults=f"{results}/zip"

    #rawresults
    rawresults=f"{results}/raw"

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(resources)

    #nobs
    nobs=len(files)

    #tot
    tot=nobs-1

    #for
    for i, file in enumerate(files):

        #ZipFile
        filepath=f"{file}"
        with ZipFile(filepath, "r") as myzip:

            #namelist
            namelist=myzip.namelist()

            #namelist
            namelist=[x for x in namelist if x.endswith(".txt") or x.endswith(".TXT")]

            #for
            for j, tempfile in enumerate(namelist):

                #stem
                tempfilestem=Path(tempfile).stem

                #lower
                tempfilestem=tempfilestem.lower()

                #extract
                member=f"{tempfile}"
                path=f"{zipresults}"
                myzip.extract(member, path)

                #rename
                p=Path(f"{zipresults}/{tempfile}")
                target=f"{rawresults}/{tempfilestem}.txt"
                p.rename(target)

        #print
        print(f"{i}/{tot} - {file} - done")

    #'''
#_faers_extract(folders, items)


#_faers_demo
#https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html
folders=["ciani/data/faers", "ciani/_faers"]
items=["demo"]
def _faers_demo(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"

    #cleanresults
    cleanresults=f"{results}/clean"

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(rawresults)

    #filestems_type
    filestems=[x for x in filestems if x.startswith(result)]
    
    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1
  
    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "PRIMARYID", #Unique number for identifying a FAERS report
        "CASEID", #Number for identifying a FAERS case
        "CASEVERSION", #Safety Report Version Number
        "I_F_CODE", #Code for initial or follow-up status of report
        "EVENT_DT", #Date the adverse event occurred or began
        "MFR_DT", #Date manufacturer first received initial information
        "INIT_FDA_DT", #Date FDA received first version
        "FDA_DT", #Date FDA received Case
        "REPT_COD", #report type: "EXP" Expedited (15-Day), "PER" Periodic (Non-Expedited), "DIR" Direct, "5DAY" 5-Day, "30DAY" 30-Day
        "AUTH_NUM", #Regulatory Authority’s case report number
        "MFR_NUM", #Manufacturer's unique report identifier
        "MFR_SNDR", #Coded name of manufacturer sending report
        "LIT_REF", #Literature Reference information
        "AGE", #patient's age at event
        "AGE_COD", #Unit abbreviation for patient's age: "DEC" DECADE, "YR" YEAR, "MON" MONTH, "WK" WEEK, "DY" DAY, "HR" HOUR
        "AGE_GRP", #Patient Age Group code: "N" Neonate, "I" Infant, "C" Child, "T" Adolescent, "A" Adult, "E" Elderly
        "SEX", # patient's sex: "UNK" Unknown, "M" Male, "F" Female
        "E_SUB", #Whether (Y/N) this report was submitted under the electronic submissions
        "WT", #patient's weight
        "WT_COD", #Unit abbreviation for patient's weight: "KG" Kilograms, "LBS" Pounds, "GMS" Grams
        "REPT_DT", #Date report was sent (YYYYMMDD format)
        "TO_MFR", #Whether (Y/N) voluntary reporter also notified manufacturer
        "OCCP_COD", #reporter's type of occupation: "MD" Physician, "PH" Pharmacist, "OT" Other health-professional, "LW" Lawyer, "CN" Consumer
        "REPORTER_COUNTRY", #country of the reporter: https://evs.nci.nih.gov/ftp1/GENC/NCIt-GENC_Terminology.txt
        "OCCR_COUNTRY", #country where the event occurred
        ]
    
    #excludebefore2014
    excludebefore2014=[
        "AUTH_NUM",
        "LIT_REF",
        "AGE_GRP",
        ]
    
    #dropna
    dropna_cols=[
        "PRIMARYID",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "PRIMARYID",
        ]

    #ordered
    ordered_cols=[
        "PRIMARYID",
        "CASEID",
        "CASEVERSION",
        "I_F_CODE",
        "EVENT_DT",
        "MFR_DT",
        "INIT_FDA_DT",
        "FDA_DT",
        "REPT_COD",
        "AUTH_NUM",
        "MFR_NUM",
        "MFR_SNDR",
        "LIT_REF",
        "AGE",
        "AGE_COD",
        "AGE_GRP",
        "SEX",
        "E_SUB",
        "WT",
        "WT_COD",
        "REPT_DT",
        "TO_MFR",
        "OCCP_COD",
        "REPORTER_COUNTRY",
        "OCCR_COUNTRY",
        "year",
        "quarter",
        ]

    #for
    for i, filestem in enumerate(filestems):
            
        #year
        year=int(f"20{filestem[-4:-2]}")

        #quarter
        quarter=int(filestem[-1:])

        #if
        if year<=2014 and quarter<=2:

            #usecols
            usecols=[x for x in usecols if x not in excludebefore2014]

        #output_path
        output_path=Path(f"{cleanresults}/{filestem}.txt")

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{cleanresults}/{filestem}.csv"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=1000,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i} - {filestem} - already done")

        #elif
        elif not output_path.is_file():

            #read_csv
            filepath=f"{rawresults}/{filestem}.txt"
            sep="$"
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                #usecols=usecols,
                dtype="string",
                nrows=1000,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )
            
            #upper
            list_columns=list(df_i.columns)
            df_i.columns=[x.upper() for x in list_columns]

            #usecols
            df_i=df_i[usecols]
            
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #year
            df_i["year"]=year

            #quarter
            df_i["quarter"]=quarter

            #sortvalues
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{cleanresults}/{filestem}.csv"
            df_i.to_csv(filepath, index=False) 

            #print
            print(f"{i}/{tot} - {filestem} - done")

        #frames
        frames[i]=df_i

    #concat
    df=pd.concat(frames)

    #sortvalues

    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False) 
    #'''
#_faers_demo(folders, items)


#_faers_drug
#https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html
folders=["ciani/data/faers", "ciani/_faers"]
items=["drug"]
def _faers_drug(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"

    #cleanresults
    cleanresults=f"{results}/clean"

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(rawresults)

    #filestems_type
    filestems=[x for x in filestems if x.startswith(result)]
    
    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1
  
    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "PRIMARYID", #Unique number for identifying a FAERS report
        "CASEID", #Number for identifying a FAERS case
        "DRUG_SEQ", #Unique number for identifying a drug for a Case
        "ROLE_COD", #drug's reported role in event: "PS" Primary Suspect Drug, "SS" Secondary Suspect Drug, "C" Concomitant, "I" Interacting
        "DRUGNAME", #Name of medicinal product
        "PROD_AI", #Product Active Ingredient
        "VAL_VBM", #source of DRUGNAME: "1" Validated trade name used, "2" Verbatim name used
        "ROUTE", #The route of drug administration
        "DOSE_VBM", #Verbatim text for dose, frequency, and route
        "CUM_DOSE_CHR", #Cumulative dose to first reaction
        "CUM_DOSE_UNIT", #Cumulative dose to first reaction unit
        "DECHAL", #reaction abated when drug therapy was stopped: "Y" Positive dechallenge, "N" Negative dechallenge, "U" Unknown, "D" Does not apply
        "RECHAL", #reaction recurred when drug therapy was restarted: "Y" Positive rechallenge, "N" Negative rechallenge, "U" Unknown, "D" Does not apply
        "LOT_NUM", #Lot number of the drug
        "EXP_DT", #Expiration date of the drug
        "NDA_NUM", #NDA number
        "DOSE_AMT", #Amount of drug reported
        "DOSE_UNIT", #Unit of drug dose
        "DOSE_FORM", #Form of dose reported
        "DOSE_FREQ", #Code for Frequency
        ]
    
    #excludebefore2014
    excludebefore2014=[
        "PROD_AI",
        ]
    
    #dropna
    dropna_cols=[
        "PRIMARYID",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "PRIMARYID",
        "DRUG_SEQ",
        ]

    #ordered
    ordered_cols=[
        "PRIMARYID",
        "CASEID",
        "DRUG_SEQ",
        "ROLE_COD",
        "DRUGNAME",
        "PROD_AI",
        "VAL_VBM",
        "ROUTE",
        "DOSE_VBM",
        "CUM_DOSE_CHR",
        "CUM_DOSE_UNIT",
        "DECHAL",
        "RECHAL",
        "LOT_NUM",
        "EXP_DT",
        "NDA_NUM",
        "DOSE_AMT",
        "DOSE_UNIT",
        "DOSE_FORM",
        "DOSE_FREQ",
        "year",
        "quarter",
        ]

    #for
    for i, filestem in enumerate(filestems):
            
        #year
        year=int(f"20{filestem[-4:-2]}")

        #quarter
        quarter=int(filestem[-1:])

        #if
        if year<=2014 and quarter<=2:

            #usecols
            usecols=[x for x in usecols if x not in excludebefore2014]

        #output_path
        output_path=Path(f"{cleanresults}/{filestem}.txt")

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{cleanresults}/{filestem}.csv"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                nrows=1000,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i} - {filestem} - already done")

        #elif
        elif not output_path.is_file():

            #read_csv
            filepath=f"{rawresults}/{filestem}.txt"
            sep="$"
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                #usecols=usecols,
                dtype="string",
                #nrows=1000,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )
            
            #upper
            list_columns=list(df_i.columns)
            df_i.columns=[x.upper() for x in list_columns]

            #usecols
            df_i=df_i[usecols]
            
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #year
            df_i["year"]=year

            #quarter
            df_i["quarter"]=quarter

            #sortvalues
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{cleanresults}/{filestem}.csv"
            df_i.to_csv(filepath, index=False) 

            #print
            print(f"{i}/{tot} - {filestem} - done")

        #frames
        frames[i]=df_i

    #concat
    df=pd.concat(frames)

    #sortvalues

    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False) 
    #'''
#_faers_drug(folders, items)


#_faers_outc
#https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html
folders=["ciani/data/faers", "ciani/_faers"]
items=["outc"]
def _faers_outc(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"

    #cleanresults
    cleanresults=f"{results}/clean"

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(rawresults)

    #filestems_type
    filestems=[x for x in filestems if x.startswith(result)]
    
    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1
  
    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "PRIMARYID", #Unique number for identifying a FAERS report
        "CASEID", #Number for identifying a FAERS case
        "OUTC_COD", #patient outcome: "DE" Death, "LT" Life-Threatening, "HO" Hospitalization, "DS" Disability, "CA" Congenital Anomaly, "RI" Required Intervention, "OT" Other Serious     
        ]
    
    #excludebefore2014
    excludebefore2014=[
        ]
    
    #dropna
    dropna_cols=[
        "PRIMARYID",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "PRIMARYID",
        "year",
        "quarter",
        ]

    #ordered
    ordered_cols=[
        "PRIMARYID",
        "CASEID",
        "OUTC_COD",
        "year",
        "quarter",
        ]

    #for
    for i, filestem in enumerate(filestems):
            
        #year
        year=int(f"20{filestem[-4:-2]}")

        #quarter
        quarter=int(filestem[-1:])

        #if
        if year<=2014 and quarter<=2:

            #usecols
            usecols=[x for x in usecols if x not in excludebefore2014]

        #output_path
        output_path=Path(f"{cleanresults}/{filestem}.txt")

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{cleanresults}/{filestem}.csv"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=1000,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i} - {filestem} - already done")

        #elif
        elif not output_path.is_file():

            #read_csv
            filepath=f"{rawresults}/{filestem}.txt"
            sep="$"
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                #usecols=usecols,
                dtype="string",
                #nrows=1000,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )
            
            #upper
            list_columns=list(df_i.columns)
            df_i.columns=[x.upper() for x in list_columns]

            #usecols
            df_i=df_i[usecols]
            
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #year
            df_i["year"]=year

            #quarter
            df_i["quarter"]=quarter

            #sortvalues
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{cleanresults}/{filestem}.csv"
            df_i.to_csv(filepath, index=False) 

            #print
            print(f"{i}/{tot} - {filestem} - done")

        #frames
        frames[i]=df_i

    #concat
    df=pd.concat(frames)

    #sortvalues

    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False) 
    #'''
#_faers_outc(folders, items)


#_whoatc
#https://www.whocc.no/atc_ddd_index/
#https://www.whocc.no/filearchive/publications/2023_guidelines_web.pdf
folders=["ciani/data/whoatc", "ciani/_whoatc"]
items=["2023 ATC Index with DDDs_electronic version", "whoatc"]
atccodes=("a10a", "a10b")
def _whoatc(folders, items, atccode):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "ATC code",
        "ATC level name",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "ATC code",
        "ATC level name",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs
    df=df[df["ATC code"].apply(len)==7]

    #sortvalues

    #ordered
    ordered_cols=[
        "ATC code",
        "ATC level name",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False) 

    #startswith
    df=df[df["ATC code"].str.startswith(atccodes)] 

    #to_csv
    filepath=f"{results}/{result}_atccodes.csv"
    df.to_csv(filepath, index=False) 

    #tolist
    values_tolist=df["ATC level name"].tolist()

    #set
    atclevelnames=set(values_tolist)

    #return
    return atclevelnames 
    #'''
#_whoatc(folders, items, atccodes)


#_medicarepartd
#https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider-and-drug
#https://data.cms.gov/resources/medicare-part-d-prescribers-by-provider-and-drug-data-dictionary
#https://data.cms.gov/provider-characteristics/medicare-provider-supplier-enrollment/medicare-provider-and-supplier-taxonomy-crosswalk
folders=["ciani/data/medicarepartd", "ciani/_medicarepartd"]
items=["medicarepartd"]
def _medicarepartd(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"

    #_folder_to_filestems
    files, filestems =_folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "Prscrbr_NPI",
        "Prscrbr_City",
        "Prscrbr_State_Abrvtn",
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Clms",
        #"Tot_Day_Suply",
        "Tot_Drug_Cst",
        #"Tot_Benes",
        ]
    
    #dropna
    dropna_cols=[
        "Prscrbr_NPI",
        "Prscrbr_City",
        "Prscrbr_State_Abrvtn",
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Clms",
        #"Tot_Day_Suply",
        "Tot_Drug_Cst",
        #"Tot_Benes",
        ]

    #sortvalues
    sortvalues_cols=[
        "Prscrbr_NPI",
        "Brnd_Name",
        "Gnrc_Name",
        ]
    
    #ordered
    ordered_cols=[
        "Prscrbr_NPI",
        "Prscrbr_City",
        "Prscrbr_State_Abrvtn",
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Clms",
        #"Tot_Day_Suply",
        "Tot_Drug_Cst",
        #"Tot_Benes",
        "statename",
        "year",
        ]

    #_whoatc
    folders=["ciani/data/whoatc", "ciani/_whoatc"]
    items=["2023 ATC Index with DDDs_electronic version", "whoatc"]
    atccodes=("a10a", "a10b")
    atclevelnames=_whoatc(folders, items, atccodes)

    #for
    for i, filestem in enumerate(filestems):

        #year
        twodigityear = filestem[(filestem.find("DY") + 2) : (filestem.find("DY")+ 4)]
        year=int(f"20{twodigityear}")

        #rawfile
        rawfile=f"{rawresults}/{year}.csv"
        
        #output_path
        output_path=Path(rawfile)

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{rawfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                #usecols=usecols,
                dtype="string",
                #nrows=1000,
                encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i} - {filestem} - already done")

        #elif
        elif not output_path.is_file():


            #read_csv
            filepath=f"{resources}/{filestem}.csv"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=1000,
                encoding="latin-1",
                #on_bad_lines="skip",
                )
            
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #dropobs
            df_i=df_i[df_i["Gnrc_Name"].isin(atclevelnames)]

            #statename
            df_i["statename"]=df_i["Prscrbr_State_Abrvtn"].replace(statenames_dict)

            #year
            df_i["year"]=year

            #sortvalues
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{rawfile}"
            df_i.to_csv(filepath, index=False)

            #print
            print(f"{i} - {filestem} - done")

        #frames
        frames[i]=df_i

    #concat
    df=pd.concat(frames)

    #sortvalues
    
    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarepartd(folders, items)


#_cmsndf
#https://data.cms.gov/provider-data/dataset/mj5m-pzi6
folders=["ciani/data/cmsndf", "ciani/_cmsndf"]
items=["DAC_NationalDownloadableFile", "cmsndf"]
def _cmsndf(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "NPI",
        "adr_ln_1",
        "City/Town",
        "State",
        "ZIP Code",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "NPI",
        "adr_ln_1",
        "City/Town",
        "State",
        "ZIP Code",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs

    #dropdups
    dropdups_cols=[
        "NPI",
        "City/Town",
        "State",
        ]
    df=df.drop_duplicates(subset=dropdups_cols)
    
    #zipcode
    df["zipcode"]=df["ZIP Code"].str[0:5]

    #sortvalues
    sortvalues_cols=[
        "NPI",
        "State",
        "City/Town", 
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "NPI",
        "adr_ln_1",
        "City/Town",
        "State",
        "zipcode",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_cmsndf(folders, items)


#_medicarelandscapepdp
#https://www.cms.gov/medicare/coverage/prescription-drug-coverage
folders=["ciani/data/medicarelandscapepdp", "ciani/_medicarelandscapepdp"]
items=["medicarelandscapepdp"]
def _medicarelandscapepdp(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"    

    #usecols
    usecols=[
        "State",
        "Monthly Drug Premium",
        "Annual Drug Deductible",
        "Contract ID",
        "Plan ID",
        ]
    
    #dropna
    dropna_cols=[
        "State",
        "Contract ID",
        "Plan ID",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "State",
        "Contract ID",
        "Plan ID",
        ]
    
    #ordered
    ordered_cols=[
        "State",
        "Monthly Drug Premium",
        "Annual Drug Deductible",
        "Contract ID",
        "Plan ID",
        "year",
        ]

    #_folder_to_filestems
    files, filestems =_folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #ordered
    frames=[None]*nobs

    #for
    for i, filestem in enumerate(filestems):

        #filepath
        filepath=f"{resources}/{filestem}.csv"

        #rawfile
        rawfile=f"{rawresults}/{filestem}.csv"

        #year
        year=int(filestem[0:4])
    
        #read_csv
        sep=","
        df_i=pd.read_csv(
            filepath, 
            sep=sep,
            header=3,
            usecols=usecols,
            dtype="string",
            #nrows=10,
            encoding="latin-1",
            #on_bad_lines="skip",
            )
            
        #lowercase
        df_i=_lowercase(df_i)

        #dropna
        df_i=df_i.dropna(subset=dropna_cols)

        #dropobs

        #replace
        replace_cols=[
            "Monthly Drug Premium",
            "Annual Drug Deductible",
            ]
        for j, col in enumerate(replace_cols):
            df_i[col]=df_i[col].str.strip()
            df_i[col]=df_i[col].str.replace("$", "")
            df_i[col]=df_i[col].str.replace("-", "0")

        #year
        df_i["year"]=year

        #sortvalues
        df_i=df_i.sort_values(
            by=sortvalues_cols,
            )

        #ordered
        df_i=df_i[ordered_cols]

        #to_csv
        filepath=f"{rawfile}"
        df_i.to_csv(filepath, index=False)

        #frames
        frames[i]=df_i

        #print
        print(f"{i} - {filestem} - done")

    #concat
    df=pd.concat(frames)

    #sortvalues
    
    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarelandscapepdp(folders, items)


#_medicarelandscapema
#https://www.cms.gov/medicare/coverage/prescription-drug-coverage
folders=["ciani/data/medicarelandscapema", "ciani/_medicarelandscapema"]
items=["medicarelandscapema"]
def _medicarelandscapema(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"    

    #usecols
    usecols=[
        "State",
        "County",
        "Monthly Consolidated Premium (Includes Part C + D)",
        "Annual Drug Deductible",
        "Drug Benefit Type",
        "Contract ID",
        "Plan ID",
        ]
    
    #dropna
    dropna_cols=[
        "State",
        "County",
        "Contract ID",
        "Plan ID",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "State",
        "County",
        "Contract ID",
        "Plan ID",
        "year",
        ]
    
    #ordered
    ordered_cols=[
        "State",
        "County",
        "Monthly Consolidated Premium (Includes Part C + D)",
        "Annual Drug Deductible",
        "Drug Benefit Type",
        "Contract ID",
        "Plan ID",
        "year",
        ]

    #_folder_to_filestems
    files, filestems =_folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #ordered
    frames=[None]*nobs

    #for
    for i, filestem in enumerate(filestems):

        #filepath
        filepath=f"{resources}/{filestem}.csv"

        #rawfile
        rawfile=f"{rawresults}/{filestem}.csv"
    
        #read_csv
        sep=","
        df_i=pd.read_csv(
            filepath, 
            sep=sep,
            header=5,
            #usecols=usecols,
            dtype="string",
            #nrows=10,
            encoding="latin-1",
            #on_bad_lines="skip",
            )
        
        #strip
        df_i.columns=df_i.columns.str.strip()

        #replace
        df_i.columns=df_i.columns.str.replace("\n", "")
          
        #usecols
        df_i=df_i[usecols]
        
        #lowercase
        df_i=_lowercase(df_i)

        #dropna
        df_i=df_i.dropna(subset=dropna_cols)

        #dropobs
        df_i=df_i[df_i["Annual Drug Deductible"].notna()]

        #replace
        replace_cols=[
            "Monthly Consolidated Premium (Includes Part C + D)",
            "Annual Drug Deductible",
            ]
        for j, col in enumerate(replace_cols):
            df_i[col]=df_i[col].str.strip()
            df_i[col]=df_i[col].str.replace("$", "")
            df_i[col]=df_i[col].str.replace("-", "0")

        #year
        year=int(filestem[0:4])
        df_i["year"]=year

        #sortvalues
        df_i=df_i.sort_values(
            by=sortvalues_cols,
            )

        #ordered
        df_i=df_i[ordered_cols]

        #to_csv
        filepath=f"{rawfile}"
        df_i.to_csv(filepath, index=False)

        #frames
        frames[i]=df_i

        #print
        print(f"{i} - {filestem} - done")

    #concat
    df=pd.concat(frames)

    #sortvalues
    df=df.sort_values(
        by=sortvalues_cols,
        )
    
    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarelandscapema(folders, items)


#_medicareenrollmentpdp
#https://www.cms.gov/data-research/statistics-trends-and-reports/medicare-advantagepart-d-contract-and-enrollment-data/monthly-pdp-enrollment-state/county/contract
folders=["ciani/data/medicareenrollmentpdp", "ciani/_medicareenrollmentpdp"]
items=["medicareenrollmentpdp"]
def _medicareenrollmentpdp(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #zipresults
    zipresults=f"{results}/zip"

    #rawresults
    rawresults=f"{results}/raw"

    #cleanresults
    cleanresults=f"{results}/clean"

    #lenstem
    lenstem=len("SCC_Enrollment_PDP_")

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1

    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "Contract ID",
        "FIPS Code",
        "Enrolled",
        ]

    #dropna
    dropna_cols=[
        "Contract ID",
        "FIPS Code",
        "Enrolled",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "FIPS Code",
        "Contract ID",
        "Enrolled",
        ]
    
    #ordered
    ordered_cols=[
        "Contract ID",
        "FIPS Code",
        "Enrolled",
        "year",
        "month",
        ]

    #for
    for i, filestem in enumerate(filestems):

        #zipfile
        zipfile=f"{resources}/{filestem}.zip"

        #year
        year=filestem[(lenstem):(lenstem+4)]

        #month
        month=filestem[(lenstem+5):(lenstem+5+2)]

        #rawfile
        rawfile=f"{rawresults}/{year}_{month}.csv"

        #cleanfile
        cleanfile=f"{cleanresults}/{year}_{month}.csv"

        #output_path
        output_path=Path(rawfile)

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{cleanfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=10,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i}/{tot} - {filestem} - already done") 

        #elif
        elif not output_path.is_file():

            #ZipFile
            with ZipFile(zipfile, "r") as zip_ref:

                #namelist
                namelist=zip_ref.namelist()

                #tempfile
                tempfile=[x for x in namelist if x.endswith(".csv")][0]
                
                #extract
                zip_ref.extract(tempfile, zipresults)
            
            #p
            p=Path(f"{zipresults}/{tempfile}")

            #rename
            p.rename(rawfile)

            #read_csv
            filepath=f"{rawfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=10,
                #encoding="latin-1",
                #on_bad_lines="skip",
                ) 
        
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #dropobs

            #year
            df_i["year"]=year

            #month
            df_i["month"]=month

            #sortvalues
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{cleanfile}"
            df_i.to_csv(filepath, index=False)

            #frames
            frames[i]=df_i

            #print
            print(f"{i}/{tot} - {filestem} - done")  

    #concat
    df=pd.concat(frames)

    #sortvalues
    
    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicareenrollmentpdp(folders, items)


#_medicarepenetrationpdp
#https://www.cms.gov/data-research/statistics-trends-and-reports/medicare-advantagepart-d-contract-and-enrollment-data/pdp-state/county-penetration
folders=["ciani/data/medicarepenetrationpdp", "ciani/_medicarepenetrationpdp"]
items=["medicarepenetrationpdp"]
def _medicarepenetrationpdp(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #zipresults
    zipresults=f"{results}/zip"

    #rawresults
    rawresults=f"{results}/raw"

    #cleanresults
    cleanresults=f"{results}/clean"

    #lenstem
    lenstem=len("State_County_Penetration_PDP_")

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1

    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "FIPS",
        "Eligibles",
        "Enrolled",
        ]

    #dropna
    dropna_cols=[
        "FIPS",
        "Eligibles",
        "Enrolled",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "FIPS",
        ]

    #ordered
    ordered_cols=[
        "FIPS",
        "Eligibles",
        "Enrolled",
        "year",
        "month",
        ]

    #for
    for i, filestem in enumerate(filestems):

        #zipfile
        zipfile=f"{resources}/{filestem}.zip"

        #year
        year=filestem[(lenstem):(lenstem+4)]

        #month
        month=filestem[(lenstem+5):(lenstem+5+2)]

        #rawfile
        rawfile=f"{rawresults}/{year}_{month}.csv"

        #cleanfile
        cleanfile=f"{cleanresults}/{year}_{month}.csv"

        #output_path
        output_path=Path(rawfile)

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{cleanfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=10,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i}/{tot} - {filestem} - already done") 

        #elif
        elif not output_path.is_file():

            #ZipFile
            with ZipFile(zipfile, "r") as zip_ref:

                #namelist
                namelist=zip_ref.namelist()

                #tempfile
                tempfile=[x for x in namelist if x.endswith(".csv")][0]
                
                #extract
                zip_ref.extract(tempfile, zipresults)
            
            #p
            p=Path(f"{zipresults}/{tempfile}")

            #rename
            p.rename(rawfile)
            
            #read_csv
            filepath=f"{rawfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=10,
                #encoding="latin-1",
                #on_bad_lines="skip",
                ) 
        
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #dropobs

            #year
            df_i["year"]=year

            #month
            df_i["month"]=month

            #sortvalues
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            ordered_cols=[
                "FIPS",
                "Eligibles",
                "Enrolled",
                "year",
                "month",
                ]
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{cleanfile}"
            df_i.to_csv(filepath, index=False)

            #frames
            frames[i]=df_i

            #print
            print(f"{i}/{tot} - {filestem} - done")  

    #concat
    df=pd.concat(frames)

    #sortvalues
    
    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarepenetrationpdp(folders, items)


#_medicarepartbspending
#https://data.cms.gov/summary-statistics-on-use-and-payments/medicare-medicaid-spending-by-drug/medicare-part-b-spending-by-drug
folders=["ciani/data/medicarepartbspending", "ciani/_medicarepartbspending"]
items=["DSD_PTB_R22_P07_V10_DYT21_HCPCS", "medicarepartbspending"]
def _medicarepartbspending(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "HCPCS_Cd",
        "Brnd_Name",
        "Gnrc_Name",
        #2017
        "Tot_Spndng_2017",
        "Tot_Dsg_Unts_2017",
        "Tot_Clms_2017",
        "Tot_Benes_2017",
        #2018
        "Tot_Spndng_2018",
        "Tot_Dsg_Unts_2018",
        "Tot_Clms_2018",
        "Tot_Benes_2019",
        #2019
        "Tot_Spndng_2019",
        "Tot_Dsg_Unts_2019",
        "Tot_Clms_2019",
        "Tot_Benes_2019",
        #2020
        "Tot_Spndng_2020",
        "Tot_Dsg_Unts_2020",
        "Tot_Clms_2020",
        "Tot_Benes_2020",
        #2021
        "Tot_Spndng_2021",
        "Tot_Dsg_Unts_2021",
        "Tot_Clms_2021",
        "Tot_Benes_2021",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "HCPCS_Cd",
        "Brnd_Name",
        "Gnrc_Name",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs

    #wide_to_long
    stubnames=[
        "Tot_Spndng",
        "Tot_Dsg_Unts",
        "Tot_Clms",
        "Tot_Benes",
        ]
    i=[
        "HCPCS_Cd",
        "Brnd_Name",
        "Gnrc_Name",
        ]
    j="year"
    sep="_"
    suffix=r"\w+"
    df=pd.wide_to_long(
        df,
        stubnames=stubnames,
        i=i,
        j=j,
        sep=sep,
        suffix=suffix,
        )
    df=df.reset_index()
    
    #rename
    part="b"
    rename_dict={
        "Tot_Spndng": f"Tot_Spndng_{part}",
        "Tot_Dsg_Unts": f"Tot_Dsg_Unts_{part}",
        "Tot_Clms": f"Tot_Clms_{part}",
        "Tot_Benes": f"Tot_Benes_{part}",
        }
    df=df.rename(columns=rename_dict)

    #sortvalues
    sortvalues_cols=[
        "HCPCS_Cd",
        "year",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "HCPCS_Cd",
        "Brnd_Name",
        "Gnrc_Name",
        f"Tot_Spndng_{part}",
        f"Tot_Dsg_Unts_{part}",
        f"Tot_Clms_{part}",
        f"Tot_Benes_{part}",
        "year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarepartbspending(folders, items)


#_medicarepartbspending
#https://data.cms.gov/summary-statistics-on-use-and-payments/medicare-medicaid-spending-by-drug/medicare-part-d-spending-by-drug
folders=["ciani/data/medicarepartdspending", "ciani/_medicarepartdspending"]
items=["DSD_PTD_R22_P04_V22_D21_BGM", "medicarepartdspending"]
def _medicarepartdspending(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Mftr",
        "Mftr_Name",
        #2017
        "Tot_Spndng_2017",
        "Tot_Dsg_Unts_2017",
        "Tot_Clms_2017",
        "Tot_Benes_2017",
        #2018
        "Tot_Spndng_2018",
        "Tot_Dsg_Unts_2018",
        "Tot_Clms_2018",
        "Tot_Benes_2019",
        #2019
        "Tot_Spndng_2019",
        "Tot_Dsg_Unts_2019",
        "Tot_Clms_2019",
        "Tot_Benes_2019",
        #2020
        "Tot_Spndng_2020",
        "Tot_Dsg_Unts_2020",
        "Tot_Clms_2020",
        "Tot_Benes_2020",
        #2021
        "Tot_Spndng_2021",
        "Tot_Dsg_Unts_2021",
        "Tot_Clms_2021",
        "Tot_Benes_2021",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "Brnd_Name",
        "Gnrc_Name",
        "Mftr_Name",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs

    #wide_to_long
    stubnames=[
        "Tot_Spndng",
        "Tot_Dsg_Unts",
        "Tot_Clms",
        "Tot_Benes",
        ]
    i=[
        "Brnd_Name",
        "Gnrc_Name",
        "Mftr_Name",
        ]
    j="year"
    sep="_"
    suffix=r"\w+"
    df=pd.wide_to_long(
        df,
        stubnames=stubnames,
        i=i,
        j=j,
        sep=sep,
        suffix=suffix,
        )
    df=df.reset_index()
    
    #rename
    part="d"
    rename_dict={
        "Tot_Spndng": f"Tot_Spndng_{part}",
        "Tot_Dsg_Unts": f"Tot_Dsg_Unts_{part}",
        "Tot_Clms": f"Tot_Clms_{part}",
        "Tot_Benes": f"Tot_Benes_{part}",
        }
    df=df.rename(columns=rename_dict)
    print(df.columns)

    #sortvalues
    sortvalues_cols=[
        "Brnd_Name",
        "Mftr_Name",
        "year",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Mftr",
        "Mftr_Name",
        f"Tot_Spndng_{part}",
        f"Tot_Dsg_Unts_{part}",
        f"Tot_Clms_{part}",
        f"Tot_Benes_{part}",
        "year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarepartdspending(folders, items)


#_medicaidspending
#https://data.cms.gov/summary-statistics-on-use-and-payments/medicare-medicaid-spending-by-drug/medicaid-spending-by-drug
folders=["ciani/data/medicaidspending", "ciani/_medicaidspending"]
items=["DSD_MCD_RY24_P06_V20_D22_BGM", "medicaidspending"]
def _medicaidspending(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Mftr",
        "Mftr_Name",
        #2018
        "Tot_Spndng_2018",
        "Tot_Dsg_Unts_2018",
        "Tot_Clms_2018",
        #2019
        "Tot_Spndng_2019",
        "Tot_Dsg_Unts_2019",
        "Tot_Clms_2019",
        #2020
        "Tot_Spndng_2020",
        "Tot_Dsg_Unts_2020",
        "Tot_Clms_2020",
        #2021
        "Tot_Spndng_2021",
        "Tot_Dsg_Unts_2021",
        "Tot_Clms_2021",
        #2022
        "Tot_Spndng_2022",
        "Tot_Dsg_Unts_2022",
        "Tot_Clms_2022",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "Brnd_Name",
        "Gnrc_Name",
        "Mftr_Name",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs

    #wide_to_long
    stubnames=[
        "Tot_Spndng",
        "Tot_Dsg_Unts",
        "Tot_Clms",
        ]
    i=[
        "Brnd_Name",
        "Gnrc_Name",
        "Mftr_Name",
        ]
    j="year"
    sep="_"
    suffix=r"\w+"
    df=pd.wide_to_long(
        df,
        stubnames=stubnames,
        i=i,
        j=j,
        sep=sep,
        suffix=suffix,
        )
    df=df.reset_index()
    
    #sortvalues
    sortvalues_cols=[
        "Brnd_Name",
        "Mftr_Name",
        "year",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "Brnd_Name",
        "Gnrc_Name",
        "Tot_Mftr",
        "Mftr_Name",
        "Tot_Spndng",
        "Tot_Dsg_Unts",
        "Tot_Clms",
        "year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicaidspending(folders, items)


#_medicarepartbasp
#https://www.cms.gov/medicare/payment/fee-for-service-providers/part-b-drugs/average-drug-sales-price
#https://www.cms.gov/medicare/payment/all-fee-service-providers/medicare-part-b-drug-average-sales-price/asp-pricing-files
folders=["ciani/data/medicarepartbasp", "ciani/_medicarepartbasp"]
items=["medicarepartbasp"]
def _medicarepartbasp(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #zipresults
    zipresults=f"{results}/zip"

    #rawresults
    rawresults=f"{results}/raw"

    #cleanresults
    cleanresults=f"{results}/clean"

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1

    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "HCPCS Code",
        "HCPCS Code Dosage",
        "Payment Limit",
        ]
    
    #dropna
    dropna_cols=[
        "HCPCS Code",
        "Payment Limit",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "HCPCS Code",
        "year",
        "quarter",
        ]
    
    #ordered
    ordered_cols=[
        "HCPCS Code",
        "HCPCS Code Dosage",
        "Payment Limit",
        "year",
        "quarter",
        ]

    #for
    for i, filestem in enumerate(filestems):

        #zipfile
        zipfile=f"{resources}/{filestem}.zip"

        #year
        year=filestem.split("_")[0]

        #month
        month=filestem.split("_")[1]

        #rawfile
        rawfile=f"{rawresults}/{year}_{month}.csv"

        #cleanfile
        cleanfile=f"{cleanresults}/{year}_{month}.csv"

        #output_path
        output_path=Path(rawfile)

        #if
        if output_path.is_file():

            #read_csv
            filepath=f"{cleanfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                usecols=usecols,
                dtype="string",
                #nrows=10,
                #encoding="latin-1",
                #on_bad_lines="skip",
                )

            #print
            print(f"{i}/{tot} - {filestem} - already done") 

        #elif
        elif not output_path.is_file():

            #ZipFile
            with ZipFile(zipfile, "r") as zip_ref:

                #namelist
                namelist=zip_ref.namelist()

                #tempfile
                tempfile=[x for x in namelist if x.endswith(".csv")][0]
                
                #extract
                zip_ref.extract(tempfile, zipresults)
            
            #p
            p=Path(f"{zipresults}/{tempfile}")

            #rename
            p.rename(rawfile)

            #firstcol
            firstcol="HCPCS Code"

            #reader
            with open(rawfile, newline='') as csvfile:
                reader = csv.reader(csvfile)
                
                #for
                for j, row in enumerate(reader):
                    
                    #if
                    if firstcol in row:
                        
                        #header
                        header=j

                        #break
                        break

            #read_csv
            filepath=f"{rawfile}"
            sep=","
            df_i=pd.read_csv(
                filepath, 
                sep=sep,
                header=header,
                usecols=usecols,
                dtype="string",
                #nrows=10,
                encoding="latin-1",
                #on_bad_lines="skip",
                ) 
            
            #lowercase
            df_i=_lowercase(df_i)

            #dropna
            df_i=df_i.dropna(subset=dropna_cols)

            #dropobs

            #year
            df_i["year"]=year

            #month
            df_i["month"]=month

            #todatetime
            todatetime_cols=[
                "month",
                ]
            errors="raise"
            format="%m"
            df_i=_todatetime(df_i, todatetime_cols, errors, format)
            #quarter
            df_i["quarter"]=pd.DatetimeIndex(df_i["month"], ambiguous="NaT").quarter

            #sortvalues            
            df_i=df_i.sort_values(
                by=sortvalues_cols,
                )

            #ordered
            df_i=df_i[ordered_cols]

            #to_csv
            filepath=f"{cleanfile}"
            df_i.to_csv(filepath, index=False)

            #print
            print(f"{i}/{tot} - {filestem} - done")  

        #frames
        frames[i]=df_i

    #concat
    left=pd.concat(frames)

    #right
    filepath="ciani/_medicarepartbspending/medicarepartbspending.csv"
    sep=","
    usecols=["HCPCS_Cd", "Brnd_Name", "Gnrc_Name"]
    right=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        ) 
    right=right.drop_duplicates(subset=["HCPCS_Cd"])

    #merge
    how="left"
    left_on=["HCPCS Code"]
    right_on=["HCPCS_Cd"]
    suffixes=("_left", "_right")
    indicator=f"_merge_{result}"
    validate="m:1"
    df=pd.merge(
        left=left,
        right=right,
        how=how,
        left_on=left_on,
        right_on=right_on,
        suffixes=suffixes,
        indicator=indicator,
        validate=validate,
        )

    #sortvalues
    sortvalues_cols=[
        "HCPCS Code",
        "year",
        "quarter",
        ]          
    df=df.sort_values(
        by=sortvalues_cols,
        )
    
    #ordered
    ordered_cols=[
        "HCPCS Code",
        "HCPCS Code Dosage",
        "Payment Limit",
        "year",
        "quarter",
        "Brnd_Name",
        "Gnrc_Name",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicarepartbasp(folders, items)


#_medicaidnadac
#https://data.medicaid.gov/nadac
folders=["ciani/data/medicaidnadac", "ciani/_medicaidnadac"]
items=["medicaidnadac"]
def _medicaidnadac(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    #resource=items[0]
    result=items[0]

    #rawresults
    rawresults=f"{results}/raw"

    #_folder_to_filestems
    files, filestems = _folder_to_filestems(resources)

    #nobs
    nobs=len(filestems)

    #tot
    tot=nobs-1

    #ordered
    frames=[None]*nobs

    #usecols
    usecols=[
        "ndc_description",
        "ndc",
        "nadac_per_unit",
        ]
    
    #dropna
    dropna_cols=[
        "ndc_description",
        "ndc",
        "nadac_per_unit",
        ]
    
    #sortvalues
    sortvalues_cols=[
        "ndc_description",
        "ndc",
        ]
    
    #ordered
    ordered_cols=[
        "ndc_description",
        "ndc",
        "nadac_per_unit",
        "year",
        ]

    #for
    for i, filestem in enumerate(filestems):

        #year
        year=f"{filestem}"

        #rawfile
        rawfile=f"{rawresults}/{filestem}.csv"

        #filepath
        filepath=f"{resources}/{filestem}.csv"

        #read_csv
        sep=","
        df_i=pd.read_csv(
            filepath, 
            sep=sep,
            usecols=usecols,
            dtype="string",
            #nrows=10,
            #encoding="latin-1",
            #on_bad_lines="skip",
            ) 
            
        #lowercase
        df_i=_lowercase(df_i)

        #dropna
        df_i=df_i.dropna(subset=dropna_cols)

        #dropobs

        #year
        df_i["year"]=year

        #tonumeric
        tonumeric_cols=[
            "nadac_per_unit",
            ]
        errors="raise"
        df_i=_tonumeric(df_i, tonumeric_cols, errors)

        #_groupby
        by=[
            "ndc_description",
            "year",
            ]
        dict_agg_colfunctions={
            "ndc": _firstvalue_join_notna,
            "nadac_per_unit": ["mean"],
            }
        df_i=_groupby(df_i, by, dict_agg_colfunctions)

        #sortvalues            
        df_i=df_i.sort_values(
            by=sortvalues_cols,
            )

        #ordered
        df_i=df_i[ordered_cols]

        #to_csv
        filepath=f"{rawfile}"
        df_i.to_csv(filepath, index=False)

        #frames
        frames[i]=df_i

        #print
        print(f"{i}/{tot} - {filestem} - done")  

    #concat
    df=pd.concat(frames)

    #sortvalues
    
    #ordered

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_medicaidnadac(folders, items)


#_fdaorangeproducts
#https://www.fda.gov/drugs/drug-approvals-and-databases/orange-book-data-files
folders=["ciani/data/fdaorange", "ciani/_fdaorange"]
items=["products", "products"]
def _fdaorangeproducts(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "Ingredient",
        "DF;Route",
        "Trade_Name",
        "Applicant",
        "Strength",
        "Appl_Type",
        "Appl_No",
        "Product_No",
        "TE_Code",
        "Approval_Date",
        "Type",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.txt"
    sep="~"
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "Trade_Name",
        "Approval_Date",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs
    df=df[df["Approval_Date"]!="approved prior to jan 1, 1982"]

    #dosageform route
    df[["dosageform", "route"]] = df["DF;Route"].str.split(';', expand=True)

    #strength
    pattern=r"\s*\([^)]*\)"
    df["strength"]=df["Strength"].str.replace(pattern, "", regex=True)

    #todate
    todatetime_cols=[
        "Approval_Date",
        ]
    errors="raise"
    format="%b %d, %Y"
    df=_todatetime(df, todatetime_cols, errors, format)
    #year
    df["approval_year"]=pd.DatetimeIndex(df["Approval_Date"], ambiguous="NaT").year

    #dropobs
    df=df[df["approval_year"]>=2006]
    df=df[df["approval_year"]<=2021]

    #dropdups
    dropdups_cols=[
        "Trade_Name",
        "Applicant",
        "Appl_No",
        "strength",
        "approval_year",
        ]
    #df=df.drop_duplicates(subset=dropdups_cols)
    
    #sortvalues
    sortvalues_cols=[
        "Appl_No",
        "Product_No",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "Ingredient",
        "dosageform",
        "route",
        "Trade_Name",
        "Applicant",
        "Appl_Type",
        "Appl_No",
        "Product_No",
        "TE_Code",
        "Approval_Date",
        "Type",
        "strength",
        "approval_year"
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_fdaorangeproducts(folders, items)


#_name_suffixs
def _name_suffixs(names, suffixes):

    #nobs
    nobs=len(names)

    #name_suffixs
    name_suffixs=[None]*nobs

    #for
    for i, name in enumerate(names):

        #if
        if pd.isna(name):

            #name_suffix
            name_suffix=None

        #elif
        elif not pd.isna(name):

            #suffix
            suffix=suffixes[i]

            #if
            if pd.isna(suffix):

                #name_suffix
                name_suffix=f"{name}"

            #elif
            elif not pd.isna(suffix):

                #name_suffix
                name_suffix=f"{name} {suffix}"

        #name_suffixs
        name_suffixs[i]=name_suffix

    #return
    return name_suffixs


#_num_dems
def _num_dems(numerators, denominators):

    #nobs
    nobs=len(numerators)

    #strengths
    strengths=[None]*nobs

    #for
    for i, numerator in enumerate(numerators):

        #denominator
        denominator=denominators[i]

        #if
        if pd.isna(numerator) or pd.isna(denominator):

            #strength
            strength=None

        #elif
        elif pd.notna(numerator) and pd.notna(denominator):

            #split
            numerator_list = numerator.split('; ')
            denominator_list = denominator.split('; ')

            #zip
            strength_list = [f"{x}/{y}" for x, y in zip(numerator_list, denominator_list)]

            #strength
            strength = '; '.join(strength_list)

        #strengths
        strengths[i]=strength

    #return
    return strengths


#_fdandcproduct
#https://www.fda.gov/drugs/drug-approvals-and-databases/national-drug-code-directory
#https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-product-file-definitions
folders=["ciani/data/fdandc", "ciani/_fdandc"]
items=["product", "product"]
def _fdandcproduct(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "PRODUCTID",
        "PRODUCTTYPENAME",
        "PROPRIETARYNAME",
        "PROPRIETARYNAMESUFFIX",
        "NONPROPRIETARYNAME",
        "DOSAGEFORMNAME",
        "ROUTENAME",
        "STARTMARKETINGDATE",
        "ENDMARKETINGDATE",
        "MARKETINGCATEGORYNAME",
        "APPLICATIONNUMBER",
        "LABELERNAME",
        "SUBSTANCENAME",
        "ACTIVE_NUMERATOR_STRENGTH",
        "ACTIVE_INGRED_UNIT",
        "PHARM_CLASSES",
        "NDC_EXCLUDE_FLAG",
    
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.txt"
    sep="\t"
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "APPLICATIONNUMBER",
        "STARTMARKETINGDATE",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs
    dropobs=[
        "nda",
        "anda",
        "nda authorized generic",
        ]
    df=df[df["MARKETINGCATEGORYNAME"].isin(dropobs)]
    dropobs=[
        "e",
        "u",
        "i",
        ]
    df=df[~df["NDC_EXCLUDE_FLAG"].isin(dropobs)]

    #name
    names=df["PROPRIETARYNAME"].tolist()
    suffixes=df["PROPRIETARYNAMESUFFIX"].tolist()
    name_suffixs=_name_suffixs(names, suffixes)
    df["name"]=name_suffixs

    #applno
    df["applno"]=df["APPLICATIONNUMBER"].str[-6:]

    #strength
    numerators=df["ACTIVE_NUMERATOR_STRENGTH"].tolist()
    denominators=df["ACTIVE_INGRED_UNIT"].tolist()
    strengths=_num_dems(numerators, denominators)
    df["strength"]=strengths

    #todate
    todatetime_cols=[
        "STARTMARKETINGDATE",
        ]
    errors="raise"
    format="%Y%m%d"
    df=_todatetime(df, todatetime_cols, errors, format)
    #year
    df["startmarketing_year"]=pd.DatetimeIndex(df["STARTMARKETINGDATE"], ambiguous="NaT").year

    #dropobs
    df=df[df["startmarketing_year"]>=2006]
    df=df[df["startmarketing_year"]<=2021]

    #dropdups
    dropdups_cols=[
        "LABELERNAME",
        "name",
        "applno",
        "strength",
        "startmarketing_year",
        ]
    #df=df.drop_duplicates(subset=dropdups_cols)

    #sortvalues
    sortvalues_cols=[
        "applno",
        "PRODUCTID",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "PRODUCTID",
        "PRODUCTTYPENAME",
        "NONPROPRIETARYNAME",
        "DOSAGEFORMNAME",
        "ROUTENAME",
        "STARTMARKETINGDATE",
        "ENDMARKETINGDATE",
        "MARKETINGCATEGORYNAME",
        "APPLICATIONNUMBER",
        "LABELERNAME",
        "SUBSTANCENAME",
        "PHARM_CLASSES",
        "name",
        "applno",
        "strength",
        "startmarketing_year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''


#_brands_generics
def _brands_generics(products):

    #nobs
    nobs=len(products)

    #brands
    brands=[None]*nobs

    #generics
    generics=[None]*nobs

    #comma
    comma=","

    #leftparen
    leftparen="("

    for i, product in enumerate(products):

        #replace
        product=product.replace(";", ",")
        product=product.replace("[", "(").replace("]", ")")

        #if
        if comma not in product:

            #if
            if not leftparen in product:

                #brand
                brand=product.strip()

                #generic
                generic=None

            #elif
            elif leftparen in product:

                #replace
                product=product.replace(",", "")
                product=product.replace(" and ", " ")

                #match
                pattern=r"(.+?)\s*\((.+?)\)"
                match=re.match(pattern, product)

                #brand
                brand=match.group(1).strip()

                #generic
                generic=match.group(2).strip()

        #elif
        elif comma in product:

            #comma_idx
            comma_idx = product.find(comma)

            #if
            if leftparen not in product: 

                #brand
                brand=product.split(comma)[0].strip()

                #generic
                generic=product.split(comma)[1].strip()

            #elif
            elif leftparen in product:

                #leftparen_idx
                leftparen_idx = product.find(leftparen)

                #if
                if comma_idx<leftparen_idx:

                    #sub
                    pattern=r"\s*\([^)]*\)"
                    product=re.sub(pattern, "", product)

                    #brand
                    brand=product.split(comma)[0].strip()

                    #generic
                    generic=product.split(comma)[1].strip()
            
                #elif
                elif comma_idx>leftparen_idx:

                    #replace
                    product=product.replace(",", "")
                    product=product.replace(" and ", " ")

                    #match
                    pattern=r"(.+?)\s*\((.+?)\)"
                    match=re.match(pattern, product)

                    #brand
                    brand=match.group(1).strip()

                    #generic
                    generic=match.group(2).strip()

        #brands
        brands[i]=brand

        #generics
        generics[i]=generic

    #return
    return brands, generics


#_fdaaalist
#https://www.fda.gov/drugs/nda-and-bla-approvals/accelerated-approvals
#https://www.fda.gov/drugs/postmarket-requirements-and-commitments/postmarketing-requirements-and-commitments-downloadable-database-file
folders=["ciani/data/fdaaa", "ciani/_fdaaa"]
items=["Final CDER AA Approvals as of 12-31-23_1", "list"]
def _fdaaalist(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "Application Number",
        "Proprietary  Name",
        "Established  Name",
        "Applicant",
        "FDA Received Date",
        "Accelerated Approval Date",
        "Accelerated Approval Indication",
        "Conversion-Withdrawal Status",
        "Full Approval Conversion- Withdrawal Date",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=100,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )

    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "Application Number",   
        "Accelerated Approval Date",
        ]
    df=df.dropna(subset=dropna_cols)

    #replace
    df["Application Number"]=df["Application Number"].str.replace("\n", "")

    #nda_bla
    df["nda_bla"]=df["Application Number"].str[0:3]

    #applno
    df["applno"]=df["Application Number"].str[4:(4+6)]

    #supplement
    df["supplement"]=df["Application Number"].str[(4+6):]

    #dropobs
    df=df[df["nda_bla"]=="nda"]
    
    #converted
    df["converted"]=np.where(df["Conversion-Withdrawal Status"]=="converted", 1, 0)

    #withdrawn
    df["withdrawn"]=np.where(df["Conversion-Withdrawal Status"].str.contains("withdrawn"), 1, 0)

    #converted_date
    df["converted_date"]=np.where(df["converted"]==1, df["Full Approval Conversion- Withdrawal Date"], None)

    #withdrawn_date
    df["withdrawn_date"]=np.where(df["withdrawn"]==1, df["Full Approval Conversion- Withdrawal Date"], None)

    #todate
    todatetime_cols=[
        "Accelerated Approval Date",
        ]
    errors="raise"
    format="%d/%m/%Y"
    df=_todatetime(df, todatetime_cols, errors, format)
    #year
    df["approval_year"]=pd.DatetimeIndex(df["Accelerated Approval Date"], ambiguous="NaT").year
    df["converted_year"]=pd.DatetimeIndex(df["converted_date"], ambiguous="NaT").year
    df["withdrawn_year"]=pd.DatetimeIndex(df["withdrawn_date"], ambiguous="NaT").year

    #dropobs
    df=df[df["approval_year"]>=2006]
    df=df[df["approval_year"]<=2021]

    #sortvalues
    sortvalues_cols=[
        "applno",
        "approval_year",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #dropdups
    dropdups_cols=[
        "applno",
        ]
    df=df.drop_duplicates(subset=dropdups_cols)
    
    #sortvalues
    sortvalues_cols=[
        "applno",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "Proprietary  Name",
        "Established  Name",
        "Applicant",
        "FDA Received Date",
        "Accelerated Approval Date",
        "Accelerated Approval Indication",
        "applno",
        "supplement",
        "converted_date",
        "withdrawn_date",
        "approval_year",
        "converted_year",
        "withdrawn_year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''


#_fdaaapmc
#https://www.fda.gov/drugs/nda-and-bla-approvals/accelerated-approvals
#https://www.fda.gov/drugs/postmarket-requirements-and-commitments/postmarketing-requirements-and-commitments-downloadable-database-file
folders=["ciani/data/fdaaa", "ciani/_fdaaa"]
items=["pmc_commitments", "pmc"]
def _fdaaapmc(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "CMT_DESC",
        "CMT_STATUS",
        "CMT_STATUS_DESC",
        "NDA_BLA_APPROVAL_DATE",
        "NDA_NUMBER",
        "APPLICANT",
        "PRODUCT",
        "CDER_OR_CBER",
        "SUBPART_FLAG",
        ]
    
    #read_csv
    filepath=f"{resources}/{resource}.txt"
    sep="~"
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=100,
        encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        "NDA_BLA_APPROVAL_DATE",
        "NDA_NUMBER",
        "PRODUCT",
        "SUBPART_FLAG",
        ]
    df=df.dropna(subset=dropna_cols)

    #dropobs
    df=df[df["SUBPART_FLAG"]=="h"]

    #replace_dict
    replace_dict={
        "p": "pending",
        "o": "ongoing",
        "d": "delayed",
        "t": "terminated",
        "s": "submitted",
        "f": "fulfilled",
        "r": "released"
        }
    df["cmt_status_explanation"]=df["CMT_STATUS"].replace(replace_dict)

    #_groupby
    by=[
        "NDA_NUMBER",
        ]
    dict_agg_colfunctions={
        "CMT_DESC": _firstvalue_join_notna,
        "CMT_STATUS": _firstvalue_join_notna,
        "CMT_STATUS_DESC": _firstvalue_join_notna,
        "NDA_BLA_APPROVAL_DATE": _firstvalue_join_notna,
        "APPLICANT": _firstvalue_join_notna,
        "PRODUCT": _firstvalue_join_notna,
        "CDER_OR_CBER": _firstvalue_join_notna,
        "cmt_status_explanation": _firstvalue_join_notna,
        }
    df=_groupby(df, by, dict_agg_colfunctions)

    #tolist
    products=df["PRODUCT"].tolist()

    #brands_generics
    brands, generics = _brands_generics(products)

    #brand
    df["brand"]=brands

    #generic
    df["generic"]=generics

    #todate
    todatetime_cols=[
        "NDA_BLA_APPROVAL_DATE",
        ]
    errors="raise"
    format="%m/%d/%Y %H:%M:%S"
    df=_todatetime(df, todatetime_cols, errors, format)
    #year
    df["approval_year"]=pd.DatetimeIndex(df["NDA_BLA_APPROVAL_DATE"], ambiguous="NaT").year

    #dropobs
    df=df[df["approval_year"]>=2006]
    df=df[df["approval_year"]<=2021]

    #sortvalues
    sortvalues_cols=[
        "NDA_NUMBER",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "CMT_DESC",
        "CMT_STATUS",
        "CMT_STATUS_DESC",
        "NDA_BLA_APPROVAL_DATE",
        "NDA_NUMBER",
        "APPLICANT",
        "CDER_OR_CBER",
        "cmt_status_explanation",
        "brand",
        "generic",
        "approval_year",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_fdaaapmc(folders, items)


#_applnos
def _applnos(values):

    #nobs
    nobs=len(values)

    #applnos
    applnos=[None]*nobs

    #for
    for i, value in enumerate(values):

        #if
        if pd.isna(value):

            #applno
            applno=None

        #elif
        elif not pd.isna(value):

            #len
            len_value=len(value)

            #if
            if len_value==6:

                #applno
                applno=f"{value}"

            #elif
            elif len_value==5:

                #applno
                applno=f"0{value}"

        #applnos
        applnos[i]=applno

    #return
    return applnos


#_fdanmeapproval
#https://www.fda.gov/drugs/drug-approvals-and-databases/compilation-cder-new-molecular-entity-nme-drug-and-new-biologic-approvals
folders=["ciani/data/fdanmeapproval", "ciani/_fdanmeapproval"]
items=["Compilation-of-CDER-NME-and-New-Biologic-Approvals-1985-2022", "fdanmeapproval"]
def _fdanmeapproval(folders, items):

    #folders
    resources=folders[0]
    results=folders[1]

    #items
    resource=items[0]
    result=items[1]

    #usecols
    usecols=[
        "Proprietary  Name",
        "Active Ingredient/Moiety",
        "Applicant",
        "NDA/BLA",
        " Application Number(1)",
        " Application Number(2)",
        " Application Number(3)",
        "Dosage Form(1)",
        "Route of Administration(1)",
        "Dosage Form(2)",
        "Route of Administration(2)",
        "Dosage Form(3)",
        "Route of Administration(3)",
        "FDA Approval Date",
        "Approval Year",
        "Abbreviated Indication(s)",
        "Approved Use(s)",
        "Review Designation",
        "Orphan Drug Designation",
        "Accelerated Approval",
        "Breakthrough Therapy Designation",
        "Fast Track Designation",
        "Qualified Infectious Disease Product",
        "Issued a Priority Review Voucher",
        "Redeemed a Priority Review Voucher",
        ]

    #read_csv
    filepath=f"{resources}/{resource}.csv"
    sep=","
    df=pd.read_csv(
        filepath, 
        sep=sep,
        usecols=usecols,
        dtype="string",
        #nrows=10,
        #encoding="latin-1",
        #on_bad_lines="skip",
        )
    
    #lowercase
    df=_lowercase(df)

    #dropna
    dropna_cols=[
        " Application Number(1)",
        ]
    df=df.dropna(subset=dropna_cols)

    #todate
    todatetime_cols=[
        "FDA Approval Date",
        ]
    errors="raise"
    format="%m/%d/%Y"
    df=_todatetime(df, todatetime_cols, errors, format)
    #year
    df["approval_year"]=pd.DatetimeIndex(df["FDA Approval Date"], ambiguous="NaT").year

    #dropobs
    df=df[df["NDA/BLA"]=="nda"]
    df=df[df["approval_year"]>=2006]
    df=df[df["approval_year"]<=2021]
    df=df[df["Accelerated Approval"]!="no"]

    #applnos
    values=df[" Application Number(1)"].tolist()
    applnos=_applnos(values)
    df["applno_1"]=applnos
    values=df[" Application Number(2)"].tolist()
    applnos=_applnos(values)
    df["applno_2"]=applnos
    values=df[" Application Number(3)"].tolist()
    applnos=_applnos(values)
    df["applno_3"]=applnos

    #rename
    rename_dict={
        "Dosage Form(1)": "dosage_1",
        "Dosage Form(2)": "dosage_2",
        "Dosage Form(3)": "dosage_3",
        "Route of Administration(1)": "route_1",
        "Route of Administration(2)": "route_2",
        "Route of Administration(3)": "route_3",
        }
    df=df.rename(columns=rename_dict)

    #wide_to_long
    stubnames=[
        "applno",
        "dosage",
        "route",
        ]
    i=[
        "Proprietary  Name",
        "Active Ingredient/Moiety",
        "Applicant",
        "FDA Approval Date",
        "Abbreviated Indication(s)",
        "Approved Use(s)",
        "Review Designation",
        "Orphan Drug Designation",
        "Accelerated Approval",
        "Breakthrough Therapy Designation",
        "Fast Track Designation",
        "Qualified Infectious Disease Product",
        "Issued a Priority Review Voucher",
        "Redeemed a Priority Review Voucher",
        "approval_year",
        ]
    j="instance"
    sep="_"
    suffix=r"\w+"
    df=pd.wide_to_long(
        df,
        stubnames=stubnames,
        i=i,
        j=j,
        sep=sep,
        suffix=suffix,
        )
    df=df.reset_index()

    #dropna
    dropna_cols=[
        "applno",
        ]
    df=df.dropna(subset=dropna_cols)

    #sortvalues
    sortvalues_cols=[
        "applno",
        ]
    df=df.sort_values(
        by=sortvalues_cols,
        )

    #ordered
    ordered_cols=[
        "Proprietary  Name",
        "Active Ingredient/Moiety",
        "Applicant",
        "FDA Approval Date",
        "Abbreviated Indication(s)",
        "Approved Use(s)",
        "Review Designation",
        "Orphan Drug Designation",
        "Accelerated Approval",
        "Breakthrough Therapy Designation",
        "Fast Track Designation",
        "Qualified Infectious Disease Product",
        "Issued a Priority Review Voucher",
        "Redeemed a Priority Review Voucher",
        "approval_year",
        "applno",
        "dosage",
        "route",
        ]
    df=df[ordered_cols]

    #to_csv
    filepath=f"{results}/{result}.csv"
    df.to_csv(filepath, index=False)  
    #'''
#_fdanmeapproval(folders, items)


#_medicarepartdspending
folders=["ciani/data/medicarepartdspending", "ciani/_medicarepartdspending"]
items=["DSD_PTD_R22_P04_V22_D21_BGM", "medicarepartdspending"]
#_medicarepartdspending(folders, items)


#_medicarepartbspending
folders=["ciani/data/medicarepartbspending", "ciani/_medicarepartbspending"]
items=["DSD_PTB_R22_P07_V10_DYT21_HCPCS", "medicarepartbspending"]
#_medicarepartbspending(folders, items)


#_medicarepartbasp
folders=["ciani/data/medicarepartbasp", "ciani/_medicarepartbasp"]
items=["medicarepartbasp"]
#_medicarepartbasp(folders, items)


#_fdaaalist
folders=["ciani/data/fdaaa", "ciani/_fdaaa"]
items=["Final CDER AA Approvals as of 12-31-23_1", "list"]
#_fdaaalist(folders, items)


#_fdanmeapproval
folders=["ciani/data/fdanmeapproval", "ciani/_fdanmeapproval"]
items=["Compilation-of-CDER-NME-and-New-Biologic-Approvals-1985-2022", "fdanmeapproval"]
#_fdanmeapproval(folders, items)


#fdalist_fdanmeapproval
folders=["ciani/_merged"]
items=["fdalist_fdanmeapproval"]
left_path="ciani/_fdaaa/list"
left_ons=["applno"]
right_path="ciani/_fdanmeapproval/fdanmeapproval"
right_ons=["applno"]
how="inner"
validate="1:1"
#_pd_merge(folders, items, left_path, left_ons, right_path, right_ons, how, validate)


#medicarepartbasp_medicarepartbspending
folders=["ciani/_merged"]
items=["medicarepartbasp_medicarepartbspending"]
left_path="ciani/_medicarepartbasp/medicarepartbasp"
left_ons=["HCPCS Code", "Brnd_Name", "Gnrc_Name", "year"]
right_path="ciani/_medicarepartbspending/medicarepartbspending"
right_ons=["HCPCS_Cd", "Brnd_Name", "Gnrc_Name", "year"]
how="left"
validate="m:1"
#_pd_merge(folders, items, left_path, left_ons, right_path, right_ons, how, validate)


#medicarepartbasp_medicarepartbspending_fdalist_fdanmeapproval
folders=["ciani/_merged"]
items=["medicarepartbasp_medicarepartbspending_fdalist_fdanmeapproval"]
left_path="ciani/_merged/medicarepartbasp_medicarepartbspending"
left_ons=["Brnd_Name"]
right_path="ciani/_merged/fdalist_fdanmeapproval"
right_ons=["Proprietary  Name_left"]
blocking_vars=[]
threshold=0.9
how="inner"
validate="m:1"
#_linktransformer(folders, items, left_path, left_ons, right_path, right_ons, blocking_vars, threshold, how, validate)


#medicarepartdspending_fdalist_fdanmeapproval
folders=["ciani/_merged"]
items=["medicarepartdspending_fdalist_fdanmeapproval"]
left_path="ciani/_medicarepartdspending/medicarepartdspending"
left_ons=["Brnd_Name"]
right_path="ciani/_merged/fdalist_fdanmeapproval"
right_ons=["Proprietary  Name_left"]
blocking_vars=[]
threshold=0.9
how="inner"
validate="m:1"
#_linktransformer(folders, items, left_path, left_ons, right_path, right_ons, blocking_vars, threshold, how, validate)


#fdalist_fdanmeapproval_cma
folders=["ciani/_merged"]
items=["fdalist_fdanmeapproval_cma"]
left_path="ciani/_merged/fdalist_fdanmeapproval"
left_ons=["Proprietary  Name_left"]
right_path="ciani/data/cma/cma"
right_ons=["Commercial name"]
blocking_vars=[]
threshold=0.9
how="inner"
validate="1:1"
#_linktransformer(folders, items, left_path, left_ons, right_path, right_ons, blocking_vars, threshold, how, validate)

ModuleNotFoundError: No module named 'dask'